In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

### Problem Overview

The goal is to build a model that can predict whether a patient has heart with hypertrophic cardiomyopathy based on features such as:
- lung width
- heart width
- cr ratio
- interia tensors
- inscribed circle radius
- polygon area ratio
- heart perimeter
- heart area
- lung area

Each row in the dataset represents one patient, and the target column — "Cardiomegaly" — indicates the outcome:

1 → positive diagnosis (diseased heart)

0 → negative diagnosis (healthy heart)